# Pipeline

Ce notebook est tiré du Guide Officiel de Quantopian sur les pipelines. N'oubliez pas de consulter leur documentation pour obtenir de nombreuses autres ressources intéressantes !

De nombreux algorithmes de Trading ont la structure suivante :

1. Pour chaque actif d'un ensemble connu (important), calculer N valeurs scalaires pour l'actif sur la base d'une fenêtre de données.
2. Sélectionnez un ensemble d'actifs négociables plus petit sur la base des valeurs calculées en (1).
3. Calculer les pondérations de portefeuille souhaitées sur l'ensemble des actifs sélectionnés en (2).
4. Passer des ordres pour déplacer les allocations de portefeuille actuelles de l'algorithme vers les pondérations souhaitées calculées en (3).

Il y a plusieurs défis techniques à relever pour réaliser cette opération de manière robuste. Il s'agit notamment de :

* Interroger efficacement de grands ensembles d'actifs
* effectuer des calculs sur de grands ensembles d'actifs
* traitement des ajustements (fractionnements et dividendes)
* les radiations d'actifs

Le pipeline existe pour résoudre ces défis en fournissant une API uniforme pour exprimer les calculs sur une collection diverse d'ensembles de données.

## Facteurs
Un facteur est une fonction d'un actif et d'un moment dans le temps à une valeur numérique.

Un exemple simple de facteur est le prix le plus récent d'un titre. Pour un titre et un moment précis, le prix le plus récent est un nombre. Un autre exemple est le volume moyen des transactions sur 10 jours d'un titre. Les facteurs sont le plus souvent utilisés pour attribuer des valeurs aux titres, qui peuvent ensuite être utilisées de plusieurs façons. Un facteur peut être utilisé dans chacune des procédures suivantes :
* calcul des pondérations cibles
* générer un signal alpha
* construire d'autres facteurs plus complexes
* construction de filtres

## Filtres
Un filtre est une fonction allant d'un actif et d'un moment dans le temps à un booléen.
Un exemple de filtre est une fonction indiquant si le prix d'un titre est inférieur à 10 $. En fonction d'un titre et d'un moment dans le temps, cela permet d'évaluer si le titre est vrai ou faux. Les filtres sont le plus souvent utilisés pour décrire des ensembles d'actifs à inclure ou à exclure dans un but particulier.

## Classificateurs
Un classificateur est une fonction d'un actif et d'un moment dans le temps ayant une sortie catégorielle.
Plus précisément, un classificateur produit une chaîne ou un int qui ne représente pas une valeur numérique (par exemple, un label entier tel qu'un code de secteur). Les classificateurs sont le plus souvent utilisés pour regrouper des actifs pour des transformations complexes sur des sorties de facteurs. Un exemple de classificateur est la bourse sur laquelle un actif est actuellement négocié.

In [49]:
from quantopian.pipeline import Pipeline

In [50]:
def make_pipeline():
    return Pipeline()

In [51]:
pipe = make_pipeline()

In [52]:
from quantopian.research import run_pipeline

In [53]:
result = run_pipeline(pipe,'2017-01-01','2017-01-01')

In [54]:
result.head(10)

Empty DataFrame
Columns: []
Index: [(2017-01-03 00:00:00+00:00, Equity(2 [ARNC])), (2017-01-03 00:00:00+00:00, Equity(21 [AAME])), (2017-01-03 00:00:00+00:00, Equity(24 [AAPL])), (2017-01-03 00:00:00+00:00, Equity(25 [ARNC_PR])), (2017-01-03 00:00:00+00:00, Equity(31 [ABAX])), (2017-01-03 00:00:00+00:00, Equity(39 [DDC])), (2017-01-03 00:00:00+00:00, Equity(41 [ARCB])), (2017-01-03 00:00:00+00:00, Equity(52 [ABM])), (2017-01-03 00:00:00+00:00, Equity(53 [ABMD])), (2017-01-03 00:00:00+00:00, Equity(62 [ABT]))]

In [55]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8356 entries, (2017-01-03 00:00:00+00:00, Equity(2 [ARNC])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Empty DataFrame

# Data

In [56]:
from quantopian.pipeline.data.builtin import USEquityPricing

## Facteurs

N'oubliez pas que les facteurs prennent en compte un actif et un horodatage et rendent une certaine valeur numérique.

In [57]:
from quantopian.pipeline.factors import BollingerBands,SimpleMovingAverage,EWMA

In [58]:
SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)

SimpleMovingAverage([EquityPricing<US>.close], 30)

In [59]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30
    })

In [60]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [61]:
results.head(20)

30 Day Mean Close
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500
                          Equity(21 [AAME])              3.899241
                          Equity(24 [AAPL])            113.368433
                          Equity(25 [ARNC_PR])          86.796111
                          Equity(31 [ABAX])             52.498394
                          Equity(39 [DDC])               9.523000
                          Equity(41 [ARCB])             29.969167
                          Equity(52 [ABM])              42.138239
                          Equity(53 [ABMD])            114.030167
                          Equity(62 [ABT])              38.664333
                          Equity(64 [GOLD])             15.117736
                          Equity(66 [AB])               23.119167
                          Equity(67 [ADSK])             75.960667
                          Equity(69 [ACAT])             15.839500
                          Equity(70 [VBF])              18.208480
                          Equity(76 [TAP])              97.727924
                          Equity(84 [ACET])             20.722753
                          Equity(100 [IEP])             59.281167
                          Equity(106 [ACU])             22.628433
                          Equity(110 [RAMP])            26.800333

In [62]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [63]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [64]:
results.head(10)

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   
                          Equity(41 [ARCB])             29.969167   
                          Equity(52 [ABM])              42.138239   
                          Equity(53 [ABMD])            114.030167   
                          Equity(62 [ABT])              38.664333   

                                                Latest Close  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55  
                          Equity(21 [AAME])             4.10  
                          Equity(24 [AAPL])           115.84  
                          Equity(25 [ARNC_PR])           NaN  
                          Equity(31 [ABAX])            52.74  
                          Equity(39 [DDC])              9.69  
                          Equity(41 [ARCB])            27.75  
                          Equity(52 [ABM])             40.68  
                          Equity(53 [ABMD])           112.70  
                          Equity(62 [ABT])             38.42

## Facteurs Combinés

In [65]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [66]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [67]:
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(24 [AAPL])               0.028481  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665

# Filtres et Ecrans

Les filtres prennent en compte un actif et un horodatage et renvoient un booléen

In [68]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

In [69]:
close_price_filter

NumExprFilter(expr='x_0 > (20.0)', bindings={'x_0': Latest([EquityPricing<US>.close], 1)})

In [70]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close,
        'Positive Percent Diff': perc_diff_check
    })

In [71]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(24 [AAPL])               0.028481   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(24 [AAPL])                     True  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False

## Ecrans

In [72]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=perc_diff_check)

In [73]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])         113.368433        115.84   
                          Equity(66 [AB])            23.119167         23.45   
                          Equity(69 [ACAT])          15.839500         15.02   
                          Equity(70 [VBF])           18.208480         18.49   
                          Equity(84 [ACET])          20.722753         21.97   

                                             Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])            0.028481   
                          Equity(66 [AB])              0.004578   
                          Equity(69 [ACAT])            0.009375   
                          Equity(70 [VBF])             0.011814   
                          Equity(84 [ACET])            0.039630   

                                            Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])                  True  
                          Equity(66 [AB])                    True  
                          Equity(69 [ACAT])                  True  
                          Equity(70 [VBF])                   True  
                          Equity(84 [ACET])                  True

### Inverser un écran

In [74]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=~perc_diff_check)

In [75]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   
                          Equity(39 [DDC])              9.69   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   
                          Equity(39 [DDC])               -0.015436   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False  
                          Equity(39 [DDC])                     False

## Combiner des Filtres

In [76]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    small_price = latest_close < 5
    
    final_filter = perc_diff_check & small_price
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [77]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            3.097778   
                          Equity(677 [AXAS])            2.265333   
                          Equity(1144 [LCTX])           3.531167   
                          Equity(1323 [CAW])            2.541333   
                          Equity(1546 [CIF])            2.500370   

                                               Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.40   
                          Equity(677 [AXAS])           2.56   
                          Equity(1144 [LCTX])          3.62   
                          Equity(1323 [CAW])           2.60   
                          Equity(1546 [CIF])           2.57   

                                               Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])             0.013271   
                          Equity(677 [AXAS])             0.145527   
                          Equity(1144 [LCTX])            0.065795   
                          Equity(1323 [CAW])             0.016002   
                          Equity(1546 [CIF])             0.015579   

                                              Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                   True  
                          Equity(677 [AXAS])                   True  
                          Equity(1144 [LCTX])                  True  
                          Equity(1323 [CAW])                   True  
                          Equity(1546 [CIF])                   True

# Fundamentals de Morningstar

`Fundamentals` donne un accès à la base de données fondamentale quantopienne. Basée sur les données fournies par Morningstar, `Fundamentals` fournit plus de 600 mesures d'entreprise remontant à 2002 (pour correspondre aux données de tarification de Quantopian).

Regardez la documentation de Fundamentals : https://www.quantopian.com/docs/data-reference/morningstar_fundamentals.


In [78]:
from quantopian.pipeline.data.morningstar import Fundamentals

In [79]:
def make_pipeline():
    
    my_funds = Fundamentals.market_cap.latest
    
    return Pipeline(columns={'Market Cap':my_funds})

In [80]:
results = run_pipeline(make_pipeline(),'2017-01-03','2017-01-03')

In [81]:
results.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8356 entries, (2017-01-03 00:00:00+00:00, Equity(2 [ARNC])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Data columns (total 1 columns):
Market Cap    5040 non-null float64
dtypes: float64(1)
memory usage: 130.6+ KB


In [82]:
results

Market Cap
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])         8.129387e+09
                          Equity(21 [AAME])        8.370243e+07
                          Equity(24 [AAPL])        6.175885e+11
                          Equity(25 [ARNC_PR])              NaN
                          Equity(31 [ABAX])        1.189172e+09
                          Equity(39 [DDC])         8.039806e+08
                          Equity(41 [ARCB])        7.084623e+08
                          Equity(52 [ABM])         2.268896e+09
                          Equity(53 [ABMD])        4.888170e+09
                          Equity(62 [ABT])         5.655141e+10
                          Equity(64 [GOLD])        1.862359e+10
                          Equity(66 [AB])          2.221112e+09
                          Equity(67 [ADSK])        1.647140e+10
                          Equity(69 [ACAT])        1.959759e+08
                          Equity(70 [VBF])                  NaN
                          Equity(76 [TAP])         2.090877e+10
                          Equity(84 [ACET])        6.602278e+08
                          Equity(100 [IEP])        8.490282e+09
                          Equity(106 [ACU])        8.501910e+07
                          Equity(110 [RAMP])       2.079641e+09
                          Equity(112 [ACY])        1.480530e+07
                          Equity(114 [ADBE])       5.090425e+10
                          Equity(117 [AEY])        1.793760e+07
                          Equity(122 [ADI])        2.237935e+10
                          Equity(128 [ADM])        2.626700e+10
                          Equity(149 [ADX])                 NaN
                          Equity(153 [AE])         1.672277e+08
                          Equity(154 [AEM])        9.434941e+09
                          Equity(157 [AEG])        1.166367e+10
                          Equity(161 [AEP])        3.095816e+10
...                                                         ...
                          Equity(50534 [HGV])               NaN
                          Equity(50535 [PK])                NaN
                          Equity(50537 [WRD])      1.734876e+09
                          Equity(50539 [TIG])      2.339749e+09
                          Equity(50540 [TRVG])     2.762639e+09
                          Equity(50541 [KEY_PRI])           NaN
                          Equity(50542 [JHMD])              NaN
                          Equity(50543 [COWZ])              NaN
                          Equity(50544 [KEG])      2.618865e+07
                          Equity(50546 [NURE])              NaN
                          Equity(50547 [YTRA])     3.297637e+08
                          Equity(50548 [PHIO_W])            NaN
                          Equity(50549 [BMLP])              NaN
                          Equity(50550 [WBIY])              NaN
                          Equity(50552 [FIV])               NaN
                          Equity(50554 [HEBT])     8.415000e+07
                          Equity(50555 [UUUU_WS])           NaN
                          Equity(50556 [SAB_CL])            NaN
                          Equity(50558 [INFR])              NaN
                          Equity(50559 [VVPR])     5.894486e+07
                          Equity(50560 [XK])                NaN
                          Equity(50561 [XE])                NaN
                          Equity(50562 [XH])                NaN
                          Equity(50563 [XM])                NaN
                          Equity(50564 [XD])                NaN
                          Equity(50565 [XI])                NaN
                          Equity(50566 [XU])                NaN
                          Equity(50567 [XF])                NaN
                          Equity(50568 [XS])                NaN
                          Equity(50569 [OUSM])              NaN

[8356 rows x 1 columns]

In [83]:
def make_pipeline():
    
    my_funds = Fundamentals.market_cap.latest
    
    # Entreprises d'une valeur de 500 milliards ou plus ;)
    cap_filter = my_funds > 500000000000
    
    return Pipeline(columns={'Market Cap':my_funds},
                    screen = cap_filter)

In [84]:
results = run_pipeline(make_pipeline(),'2017-01-03','2017-01-03')

In [85]:
results

Market Cap
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])       6.175885e+11
                          Equity(26578 [GOOG_L])  5.461871e+11
                          Equity(46631 [GOOG])    5.319681e+11

# Masques

Parfois, nous voulons ignorer certains atouts lors du calcul des expressions des pipelines. Il y a deux cas courants où il est utile d'ignorer des actifs :
* Nous voulons calculer une expression qui est coûteuse en termes de calcul, et nous savons que nous ne nous soucions que des résultats pour certains actifs.
* Nous voulons calculer une expression qui effectue des comparaisons entre des actifs, mais nous voulons que ces comparaisons soient effectuées uniquement par rapport à un sous-ensemble de tous les actifs. 

In [86]:
def make_pipeline():
    
    # Créer d'abord les masques pour les filtres
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Passer dans le masque
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [87]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            3.097778   
                          Equity(677 [AXAS])            2.265333   
                          Equity(1144 [LCTX])           3.531167   
                          Equity(1323 [CAW])            2.541333   
                          Equity(1546 [CIF])            2.500370   

                                               Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.40   
                          Equity(677 [AXAS])           2.56   
                          Equity(1144 [LCTX])          3.62   
                          Equity(1323 [CAW])           2.60   
                          Equity(1546 [CIF])           2.57   

                                               Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])             0.013271   
                          Equity(677 [AXAS])             0.145527   
                          Equity(1144 [LCTX])            0.065795   
                          Equity(1323 [CAW])             0.016002   
                          Equity(1546 [CIF])             0.015579   

                                              Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                   True  
                          Equity(677 [AXAS])                   True  
                          Equity(1144 [LCTX])                  True  
                          Equity(1323 [CAW])                   True  
                          Equity(1546 [CIF])                   True

In [88]:
len(results)

391

# Classificateurs

Un classificateur est une fonction d'un actif et d'un moment dans le temps à une sortie catégorielle telle qu'une chaîne de caractères ou un label d'entier.

In [89]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [90]:
morningstar_sector = Sector()

In [91]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [92]:
exchange

Latest([share_class_reference<US>.exchange_id], 1)

### Méthodes Classificateur

* eq (equals)
* isnull
* startswith

In [93]:
nyse_filter = exchange.eq('NYS')

In [94]:
def make_pipeline():
    
    # Créer d'abord les masques pour les filtres
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Classificateur
    nyse_filter = exchange.eq('NYS')
    
    # passer dans le masque
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check & nyse_filter
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [95]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])           1.960533         2.020   
                          Equity(3265 [GLF])           1.576367         1.725   
                          Equity(3645 [HOV])           2.406667         2.735   
                          Equity(4577 [LUB])           4.292333         4.270   
                          Equity(4971 [RT])            3.244000         3.240   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])            0.022120   
                          Equity(3265 [GLF])            0.162420   
                          Equity(3645 [HOV])            0.176939   
                          Equity(4577 [LUB])            0.004116   
                          Equity(4971 [RT])             0.009094   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])                  True  
                          Equity(3265 [GLF])                  True  
                          Equity(3645 [HOV])                  True  
                          Equity(4577 [LUB])                  True  
                          Equity(4971 [RT])                   True

In [96]:
len(results)

66

# Pipelines dans l'IDE Quantopian

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Stocker les résultats de notre pipeline DataFrame dans la variable context.
    context.output = pipeline_output('my_pipeline')